In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

In [2]:
import sys
sys.path.append("/home/tom/coding/ExTRApack/")
import ExTRA as ex

ExTRA correctly imported


In [3]:
#standard epochs
J1991=2448349.0 #1991.25
J2000=2451545.0
J2016=2457389.0

In [4]:
#standard model:
#at J2016

asc=20
dec=-10
parallax=200
mu_a_star=-100
mu_d=+300

standard=np.array([asc,dec,parallax,mu_a_star,mu_d])

In [49]:
#orbital elements
P=550
e=0.3
om=np.radians(0)
i=np.radians(30)
Om=np.radians(75)
T0=J1991+900 #900 days after standard 1991
a=3
orbpar=np.array([P,e,om,i,Om,T0,a])


#timestamps of measurements
t_mock=np.linspace(T0-2*P,T0+2*P,100)










#orbit
t_orb=np.linspace(T0-2*P,T0+2*P,int(P/5))
mock_orbit=ex.orbit(*orbpar,t_orb)

In [58]:
#rv
t_RV=np.linspace(T0-2*P,T0+2*P,int(P/5))

v=10

#RVdata
RV=ex.RV_comb(v,P,e,om,i,T0,a,parallax,t_RV)
#error
RV_err=np.ones(len(RV))*10



#plt.figure(figsize=(10,5))
#plt.scatter(t_RV,RV,s=10)
#plt.xlim([t_RV[0],t_RV[-1]])
#plt.grid()
#plt.show()


In [51]:
def write_RV(data):
    with open('mock_RV.txt', mode='w') as file:
        for row in data:
            file.write("{:<10}\t{:<10}\t{:<10}\n".format(row[0], row[1], row[2]))

In [52]:
RV_data=np.array([t_RV,RV,RV_err])
RV_data=np.transpose(RV_data)
write_RV(RV_data)

In [53]:
#HIP
#motions
t_HIP=t_RV

earth=ex.earth_position(t_HIP)

std=np.array(ex.standard_model(asc,dec,parallax,mu_a_star,mu_d,t_HIP,earth,tangential=True))
orb=np.array(ex.orbit(P,e,om,i,Om,T0,a,t_HIP))
orb_mock=np.array(ex.orbit(P,e,om,i,Om-1.5,T0,a,t_HIP))
#motion=std+orb




#A3,A4,A5,A6,A7,A8,A9=hip_ad

angle=np.arctan2(orb[1],orb[0])

A8=np.sqrt(orb[0]**2 + orb[1]**2)
A9=abs(np.random.normal(np.std(A8),0.5*np.std(A8),len(A8)))

#A9=np.ones(len(A8))*np.std(A8)*0.25

A5=np.zeros(len(A8))


#X and Y
A3=1*np.cos(angle)
A4=1*np.sin(angle)

#year of mesaurement
epoch=(t_HIP-J2000)/365.25 +2000

#years after HIP stand epoch
frac=epoch-1991.25
A6=frac*A3
A7=frac*A4 #to A4 corresponding poper motion factor


HIP=A3,A4,A5,A6,A7,A8,A9

hip_standard=np.zeros(5)


hip_standard[0],hip_standard[1]=ex.pos_recalc(*[asc,dec,mu_a_star,mu_d],J2016,J1991)
#hip standard epoch is J1991


hip_standard[2:5]=np.array(standard[2:5])



In [83]:
data[1:]

array([[ 1.00000e+00,  9.91910e-01, -1.26960e-01,  0.00000e+00,
        -4.88500e-01,  6.25200e-02,  2.09794e+00,  4.83900e-02],
       [ 2.00000e+00,  8.70120e-01, -4.92830e-01,  0.00000e+00,
        -3.80440e-01,  2.15480e-01,  2.11182e+00,  8.26080e-01],
       [ 3.00000e+00,  6.29540e-01, -7.76970e-01,  0.00000e+00,
        -2.40460e-01,  2.96770e-01,  2.17940e+00,  4.04170e-01],
       [ 4.00000e+00,  3.33450e-01, -9.42770e-01,  0.00000e+00,
        -1.08940e-01,  3.08010e-01,  2.31897e+00,  6.28640e-01],
       [ 5.00000e+00,  4.55000e-02, -9.98960e-01,  0.00000e+00,
        -1.23500e-02,  2.71160e-01,  2.51925e+00,  8.69970e-01],
       [ 6.00000e+00, -2.00530e-01, -9.79690e-01,  0.00000e+00,
         4.33500e-02,  2.11790e-01,  2.75402e+00,  5.09930e-01],
       [ 7.00000e+00, -3.98200e-01, -9.17300e-01,  0.00000e+00,
         6.40800e-02,  1.47620e-01,  2.99732e+00,  4.26080e-01],
       [ 8.00000e+00, -5.53440e-01, -8.32890e-01,  0.00000e+00,
         5.84800e-02,  8.80100e-0

In [86]:
data=np.array([np.arange(0,len(A3)),A3,A4,A5,A6,A7,A8,A9])
data[1:]=np.around(data[1:],5)
data=np.transpose(data)

In [87]:
def write_HIP(data):
    with open('mock_HIP.txt', mode='w') as file:
        for row in data:
            file.write("{:<8}\t{:<8}\t{:<8}\t{:<8}\t{:<8}\t{:<8}\t{:<8}\t{:<8}\n".format(
                row[0], row[1], row[2],row[3],row[4],row[5],row[6],row[7]))


In [88]:
write_HIP(data)